이전에 작성한 글에서 프로그램은 결과를 내보이는 것이 목적이었습니다.

여기서는 결과가 아닌 과정(상태)을 내보이는 프로그램을 작성하겠습니다.

아래 코드를 보겠습니다.

### 1. Lazy Execution

In [207]:
def promise(x):
    yield x


def cons_stream(a, b):
    return (a, promise(b))

In [208]:
s0 = cons_stream(1, cons_stream(2, cons_stream(3, 4)))
s0

(1, <generator object promise at 0x1057da900>)

위의 `tuple`이 가진 첫번째 원소인 `1`은 `s0`의 현재 상태입니다. 

그리고 두번째 원소는 `promise` 함수를 통해 값을 [제너레이터](https://wiki.python.org/moin/Generators)로 저장하고 있습니다.

즉, 이 `promise`는 다음 상태에 대한 __약속__일 뿐, 그에대한 계산을 하지 않습니다. 

( '약속'을 함으로써 다음 계산을 미루고 있기 때문에 이런 방법을 `Delayed execution` 또는 `Lazy execution`이라고 합니다. )

이처럼 스트림은 현재 상태를 가지고 다음 상태를 약속하는 차례열의(여기서는 `tuple`로 표현한) 연속입니다.



\* 원래 `SICP`에서는 '제너레이터'가 아닌 프로시저 실행을 지연함으로써 '약속'을 구현합니다.

간단히는, 아래처럼 구현할 수 있습니다.

```scheme
(define (delay proc)
    (lambda () proc))
```

적절한 함수를 만들어 `s0`를 한 번 쭉 뽑아보면,

In [209]:
def display_stream(s):
    if isinstance(s, tuple):
        print(s[0])
        return display_stream(next(s[1]))
    else:
        print(s)

In [210]:
display_stream(s0)

1
2
3
4


### 2. 스트림을 위한 인터페이스

다음으로, 

1. 임의의 정수 a부터 b까지($a < b$) 연속하는 스트림을 만들고,

2. 그 스트림에 특정 조건을 걸어서 필터링시키는 코드를 작성하겠습니다.

In [213]:
def stream_enumerate_interval(low, high):
    if high > low:
        return cons_stream(
            low, 
            stream_enumerate_interval(low+1, high))
    else:
        return high
    
    
def stream_filter(pred, stream):
    if not isinstance(stream, tuple):
        return stream
    elif pred(stream[0]):
        return cons_stream(
            stream[0],
            stream_filter(pred, next(stream[1])))
    else:
        return stream_filter(pred, next(stream[1]))

검증을 위해 1부터 10까지의 정수로 만들어진 스트림을 2로 나누어 떨어지는 경우만 필터링해보면,

In [214]:
s1 = stream_enumerate_interval(1, 10)
filtered_s1 = stream_filter(lambda x: x%2 == 0, s1)
display_stream(filtered_s1)

2
4
6
8
10


스트림을 위한 `map`함수를 만들면,

In [215]:
def stream_map(f, stream):
    if not isinstance(stream, tuple):
        return f(stream)
    else:
        return cons_stream(
            f(stream[0]),
            stream_map(f, next(stream[1]))
        )

여러 스트림을 받아들일 수 있도록 조금 더 개선하면,

In [8]:
def apply(op, sequence):
    if len(sequence) == 1:
        return sequence[0]
    else:
        return op(sequence[0], apply(op, sequence[1:]))
    
    
def nth_elements_extractor(sqs, index):
    if len(sqs) == 0:
        return ()
    else:
        return (sqs[0][index],) + nth_elements_extractor(sqs[1:], index)


def stream_map(f, *argstreams):
    print(argstreams)
    if len(argstreams) == 0:
        pass
    else:
        return cons_stream(
            apply(f, nth_elements_extractor(argstreams, 0)),
            apply(stream_map, (f, nth_elements_extractor(argstreams, 1))))

In [25]:
p = iter([1])
q = iter([2])
a = ((1,p), (3,q))

#apply(f, first_elements_extractor(argstreams))


# it's clue to solve the stream
a= map(lambda e: next(e), nth_elements_extractor(a, 1))
a
tuple(a)
#nth_elements_extractor(a, 1)


(1, 2)

In [319]:
#from itertools import filterfalse
from operator import add
#apply(add, (1, 2,3))
a = ((1,2), (3,4))

#apply(f, first_elements_extractor(argstreams))

a= apply(add, first_elements_extractor(a))
a
#next(a)
#next(a)

4

In [304]:
from types import GeneratorType

s2 = stream_enumerate_interval(5, 15)
s3 = stream_enumerate_interval(10, 20)
a = map(
    lambda *streams: ()
        if isinstance(streams[0], GeneratorType)
        else streams,
    s2, s3
)
tuple(a)


#isinstance(s2[1], GeneratorType)


((5, 10), ())

In [313]:
a = ((1,2), (3,4))

def first_elements_extractor(sqs):
    if len(sqs) == 0:
        return ()
    else:
        return (sqs[0][0],) + ex(sqs[1:])
ex(a)

(1, 3)

In [314]:
def accumulate(op, initial, sequence):
    if sequence == []:
        return initial
    else:
        return op(sequence[0], accumulate(op, initial, sequence[1:]))

In [ ]:
from types import GeneratorType

s2 = stream_enumerate_interval(5, 15)
s3 = stream_enumerate_interval(10, 20)
isinstance(s2, GeneratorType)

"""
a = map(
    lambda *streams: None
        if isinstance(streams, GeneratorType)
        else streams,
    s2, s3
)

# and type(e2) is not GeneratorType
a
tuple(a)
"""
#sqs = (s2, s2, s2)
#for a in sqs:
#    print(a[0])



#apply(lambda e: e+1, s2[0])
#tuple(a)

In [233]:
s2 = stream_enumerate_interval(5, 15)
s3 = stream_enumerate_interval(10, 20)
#apply(lambda e1, e2: print(e1, e2), s2)
#p = map(lambda e1, e2: next(e1)[0]+next(e2)[0] if type(e1) is GeneratorType else e1+e2, s2, s3)
#tuple(p)
#stream_map(lambda e1, e2: print(e1, e2), s2, s3)

#tuple(map(lambda e1, e2: e1+e2, [1,2],[3,4]))


def imap(function, *iterables):
    # imap(pow, (2,3,10), (5,2,3)) --> 32 9 1000
    iterables = map(iter, iterables)
    while True:
        args = [next(it) for it in iterables]
        if function is None:
            yield tuple(args)
        else:
            yield function(*args)

#tuple(imap(pow, [1,2,3], [4,5,6]))
#imap(pow, [1,2,3], [4,5,6])
a = imap(pow, [1,2,3], [4,5,6])
a
#next(a)

<generator object imap at 0x1057da240>

In [218]:
from types import GeneratorType

s2 = stream_enumerate_interval(5, 15)
#type(s2[1])
#s2[1]
isinstance(s2[1], GeneratorType)
#s2[1] is GeneratorType
#stream_map(lambda e1, e2: e1+e2, s2)

True

In [142]:

apply(lambda e1, e2: e1+e2, (1, 2, 3))

(1, 2, 3)
(2, 3)
(3,)


6

In [105]:
#next(s1[1])
s2 = stream_enumerate_interval(5, 15)
#display_stream(s2)
#next(s0[1])
display_stream(stream_map(lambda e: e+1, s2))

6
7
8
9
10
11
12
13
14
15
16


In [58]:
def apply(op, sequence):
    if sequence == ():
        pass
    else:
        return op(sequence[0], apply(op, sequence[1:]))
    
# apply 프로시저를 어떻게 구현하지
def stream_map(f, argstreams):
    if len(argstreams) == 0:
        pass
    return apply(map(lambda e1, : e[0], argstreams))

In [59]:
stream_map(lambda e: e+1, s1)

TypeError: apply() missing 1 required positional argument: 'sequence'

### 2. 무한 스트림

방금전 만든 `stream_enumerate_interval` 함수에 주는 두 인자의 값 사이의 크기를 무한히 크게 둘 수 있다면 스트림은 끝나지 않을 것입니다.

\* 예를 들면, `stream_enumerate_interval`($1, \infty$) 



In [54]:
p = (1, 2)
q = ()
len(q)

0

In [30]:
def t(init_values=()):
    for value in init_values:
        yield value
    yield 0
        
q = t((1, 2, 3))
next(q)
next(q)
next(q)
next(q)
next(q)

StopIteration: 

In [56]:
def mero_f(f):
    (
        lambda is_alreday_run, result:
            lambda:
                result = (f)
                is_already_run = True
        
                if not is_already_run:
                    return result
    )(False, False)

SyntaxError: invalid syntax (<ipython-input-56-41ac3cce4d1f>, line 5)